In [6]:
import json
import numpy as np
from skimage.io import imread, imshow
from skimage import transform
import matplotlib.pyplot as plt
import numpy as np
import cv2
import math
import pandas as pd
from pathlib import Path
import os
import open3d as o3d

In [7]:
via_bbox_label_file_path = "../labels/via_labels/08360a_3.json"
anno_register_file = "../labels/via_labels/depth_print_mapping.json"
woodblock_stl_file = "/mnt/hdd/thuonglc/mocban/data_synthesis/woodblock-gt-depth-gen/data/models_3d/08360_kho_khuon_in_1_r.stl"
# mapping_3d_point_file_path = "../labels/mapping_3d/08360_1_mapping_point.stl"
# mapping_2d_point_file_path = "../labels/via_labels/mapping-point-flip-08360a_3.json"
# image_2d = "../outputs/flip-images/flip_08360a_3.jpg"

In [8]:
def get_bboxes_from_via_anno_file(via_json_path, key_src='depth'):
    anno = read_json_file(via_json_path)
    dest_point_list = []
    src_point_list = []
    file_name_list = list(anno.keys())
    for anno_key in anno.keys():
        
        if key_src in anno_key:
            for point_info in anno[anno_key]['regions']:
                dest_point_list.append([point_info['shape_attributes']['cx'], point_info['shape_attributes']['cy']])
        else:
            for point_info in anno[anno_key]['regions']:
                src_point_list.append([point_info['shape_attributes']['cx'], point_info['shape_attributes']['cy']])
    return [src_point_list, dest_point_list]

def get_point_from_via_file(via_json_path):
    src_point_list, dest_point_list = get_bboxes_from_via_anno_file(via_json_path)
    np_src_point_list = np.asarray(src_point_list)
    np_dest_point_list = np.asarray(dest_point_list)
    return [np_src_point_list, np_dest_point_list]

def read_json_file(json_path):
    with open(json_path, 'r') as f:
        content = json.load(f)
    return content

def write_json_file(anno, json_path):
    with open(json_path, 'w') as f:
        json.dump(anno, f)

In [9]:
def mapping(anno_register_file, np_src_point_list, np_dest_point_list):
    tform = transform.estimate_transform('projective', np_src_point_list, np_dest_point_list)
    anno = read_json_file(anno_register_file)
    bboxes_list = parse_polybboxes(anno)
    np_bboxes = np.asarray(bboxes_list)
    np_dest_bboxes = []
    for poly_points in np_bboxes:
        np_dest_bboxes.append(tform(poly_points))
    return [np_dest_bboxes, np_bboxes]

        
def parse_polybboxes(anno):
    poly_points = []
    for key, anno_info in anno.items():
        poly_info_list = anno_info["regions"]
        # print(poly_info_list[0])
        for poly_info in poly_info_list:
            x_point_list = poly_info["shape_attributes"]["all_points_x"]
            y_point_list = poly_info["shape_attributes"]["all_points_y"]
            points = np.stack((x_point_list, y_point_list), axis=-1)
            poly_points.append(points)
    return poly_points

In [10]:
[np_src_point_list, np_dest_point_list] = get_point_from_via_file(anno_register_file)
dest_poly_points_list, origin_poly_points_list = mapping(via_bbox_label_file_path, np_src_point_list, np_dest_point_list)

/tmp/ipykernel_20528/1843842306.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np_bboxes = np.asarray(bboxes_list)


In [11]:
inverted_matrix = np.load("../labels/inverted_matrix/08360_1_img_inverted_matrix.npy")    

In [15]:
new_dest_poly_point_list = []
for dest_poly_points in dest_poly_points_list:
    one_z = np.ones((dest_poly_points.shape[0], 1))
    point_3d = np.c_[dest_poly_points, one_z, one_z]
    new_dest_poly_point_list.append(point_3d)

In [16]:
woodblock_bb_points_list = []
for new_dest_poly_points in new_dest_poly_point_list:
    point_3d = inverted_matrix@new_dest_poly_points.T
    point_3d = point_3d.T
    woodblock_bb_points_list.append(point_3d)

In [17]:
new_woodblock_bb_points_list = []
for woodblock_bb_points in woodblock_bb_points_list:
    woodblock_bb_points[:, 2] = 20
    new_woodblock_bb_points_list.append(woodblock_bb_points[:, :3])

In [18]:
from copy import deepcopy
low_woodblock_bb_points_list =  deepcopy(new_woodblock_bb_points_list)

In [19]:
for woodblock_bb_points in low_woodblock_bb_points_list:
    woodblock_bb_points[:, 2] = -20

In [20]:
bound_woodblock_bb_points_list = new_woodblock_bb_points_list + low_woodblock_bb_points_list

In [21]:
bound_woodblock_bb_points_list = []
for index, new_woodblock_bb_points in enumerate(new_woodblock_bb_points_list):
    bound_woodblock_bb_points_list.append(np.r_[new_woodblock_bb_points, low_woodblock_bb_points_list[index]])

In [22]:
min_x, max_x = np.min(bound_woodblock_bb_points_list[0][:, 0]), np.max(bound_woodblock_bb_points_list[0][:, 0])
min_y, max_y = np.min(bound_woodblock_bb_points_list[0][:, 1]), np.max(bound_woodblock_bb_points_list[0][:, 1])
min_z, max_z = np.min(bound_woodblock_bb_points_list[0][:, 2]), np.max(bound_woodblock_bb_points_list[0][:, 2])
min_bound = np.asarray([min_x, min_y, min_z]).reshape(3, 1)
max_bound = np.asarray([max_x, max_y, max_z]).reshape(3, 1)

In [23]:
print(min_bound, max_bound)

[[ 112.30486134]
 [-117.05013116]
 [ -20.        ]] [[ 127.21270628]
 [-104.44313653]
 [  20.        ]]


In [30]:
# pcd = o3d.geometry.PointCloud()
# pcd_points = o3d.utility.Vector3dVector(bound_woodblock_bb_points_list[10])
aligned_bb = o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)

In [31]:
bb_points = aligned_bb.get_box_points()

In [26]:
dest_poly_points_list[0]

array([[10012.34819372,  1916.16533892],
       [10072.15391833,  1897.66484758],
       [10465.78821666,  1801.89099297],
       [10551.29968302,  2128.6562769 ],
       [10543.00230905,  2200.43202875],
       [10417.71385449,  2257.66166678],
       [10333.11817451,  2230.19264795],
       [10127.62608084,  2249.51307594]])

In [27]:
def read_stl_file(pc_path):
    pc_points = o3d.io.read_triangle_mesh(str(pc_path))
    np_pc_points = np.asarray(pc_points.vertices)
    return pc_points, np_pc_points

In [28]:
pc_points, np_pc_points = read_stl_file(woodblock_stl_file)

In [33]:
# pcd = o3d.geometry.PointCloud()
# pcd_points = o3d.utility.Vector3dVector(bound_woodblock_bb_points_list[10])

pc_mesh = o3d.io.read_triangle_mesh(woodblock_stl_file)
character_pc = pc_mesh.crop(aligned_bb)
character_pc = o3d.geometry.TriangleMesh.compute_triangle_normals(character_pc)
character_pc.remove_duplicated_vertices()
o3d.io.write_triangle_mesh('./example.stl', character_pc)

True

In [38]:
np.asarray(character_pc.vertices)

array([[ 127.19604492, -107.33679199,    3.21885204],
       [ 127.20240021, -107.04689789,    3.15736365],
       [ 127.15847015, -107.23674774,    3.1570909 ],
       ...,
       [ 112.36270905, -104.63610077,    3.36133623],
       [ 112.33770752, -104.79846954,    3.36360312],
       [ 112.37815857, -104.48456573,    3.34957004]])